In [1]:
import sys
sys.path.append('/Users/leah/Columbia/courses/19summer/SocialBehavior/SocialBehaviorptc')

In [3]:
from ssm_ptc.models.hmm import HMM
from ssm_ptc.distributions.truncatednormal import TruncatedNormal
from ssm_ptc.utils import find_permutation, random_rotation, k_step_prediction

from project_ssms.coupled_momentum_interaction_observation2 import CoupledMomentumInteractionObservation, CoupledMomentumInteractionTransformation
from project_ssms.momentum_utils import filter_traj_by_speed
from project_ssms.utils import k_step_prediction_for_coupled_momentum_interaction_model

import torch
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("white")
sns.set_context("talk")

#from tqdm import trange
from tqdm import tqdm_notebook as tqdm

import time

from hips.plotting.colormaps import gradient_cmap, white_to_color_cmap
color_names = [
    "windows blue",
    "red",
    "amber",
    "faded green",
    "dusty purple",
    "orange"
    ]

colors = sns.xkcd_palette(color_names)
cmap = gradient_cmap(colors)

# data

In [4]:
import joblib

datasets_processed = joblib.load('/Users/leah/Columbia/courses/19summer/SocialBehavior/tracedata/all_data_3_1')  # a list of length 30, each is a social_dataset

session_data = datasets_processed[0].render_trajectories([3,8])
traj0 = np.concatenate((session_data), axis=1)

In [5]:
del datasets_processed
del session_data

In [6]:
f_traj = filter_traj_by_speed(traj0, q1=0.99, q2=0.99)

In [7]:
arena_xmin = 10
arena_xmax = 320

arena_ymin = -10
arena_ymax = 390

WATER = torch.tensor([50, 50], dtype=torch.float64)
FOOD = torch.tensor([270, 50], dtype=torch.float64)
NEST = torch.tensor([270, 330], dtype=torch.float64)
CORNER = torch.tensor([50, 330], dtype=torch.float64)

# model

In [8]:
momentum_lags = 30
momentum_weights = np.arange(0.55, 2.05, 0.05)

In [10]:
data = torch.tensor(f_traj, dtype=torch.float64)

In [11]:
torch.manual_seed(0)
np.random.seed(0)

bounds = np.array([[arena_xmin - 5, arena_xmax + 5], [arena_ymin - 5, arena_ymax + 5], 
                   [arena_xmin - 5, arena_xmax + 5], [arena_ymin - 5, arena_ymax + 5]])

max_v = np.array([5.0, 5.0, 5.0, 5.0])

m_factor = 2
i_factor = 2

K = 2
D = 4
Df = 10
T = 36000

observation = CoupledMomentumInteractionObservation(K=K, D=D, M=0, bounds=bounds,
                                         momentum_lags=momentum_lags,momentum_weights=momentum_weights,
                                         max_v=max_v, m_factor=m_factor, i_factor=i_factor)

model = HMM(K=K, D=D, M=0, observation=observation)
m_tran = model.observation.transformation

model.observation.mus_init = data[0] * torch.ones(K, D, dtype=torch.float64)

In [12]:
momentum_vecs = CoupledMomentumInteractionTransformation._compute_momentum_vecs(data[:-1], lags=momentum_lags)
interaction_vecs = CoupledMomentumInteractionTransformation._compute_direction_vecs(data[:-1])

In [13]:
model.log_likelihood(data, momentum_vecs=momentum_vecs, interaction_vecs=interaction_vecs)

tensor(-2114842.5381, dtype=torch.float64,
       grad_fn=<HMMNormalizerCythonBackward>)

In [14]:
num_iters = 5000
losses, opt = model.fit(data, num_iters=num_iters, lr=0.005, momentum_vecs=momentum_vecs, interaction_vecs=interaction_vecs)

iter 4990 loss 432009.55: 100%|██████████| 5000/5000 [29:32<00:00,  3.20it/s]  | 0/5000 [00:00<?, ?it/s]
